In [ ]:
import dagshub
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle
import pandas as pd
import os

In [ ]:

dagshub.init(repo_owner='Nagisa', repo_name='ml_ops_project', mlflow=True)

In [ ]:
# Charger les données à partir d'un fichier géré avec DVC
data_file = "data/iris.csv"
if not os.path.exists(data_file):
    raise FileNotFoundError(f"Le fichier {data_file} est introuvable. Assurez-vous qu'il est géré par DVC.")

data = pd.read_csv(data_file)
X = data.iloc[:, :-1].values  # Toutes les colonnes sauf la dernière
Y = data.iloc[:, -1].values   # La dernière colonne est la cible

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Début de l'expérience MLflow
with mlflow.start_run():
    # Entraîner un modèle simple RandomForest
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Prédire et évaluer les performances du modèle
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Journaliser les paramètres, métriques et le modèle dans MLflow
    mlflow.log_param("model_type", "RandomForestClassifier")
    mlflow.log_metric("accuracy", accuracy)

    # Sauvegarder le modèle dans un fichier pickle
    model_filename = "model/iris_model.pkl"
    with open(model_filename, "wb") as f:
        pickle.dump(model, f)

    print(f"Modèle sauvegardé dans le fichier: {model_filename}")

print("Modèle entraîné et sauvegardé avec succès.")